Reviewing the Python metaclass 
This is a calss that creates other classes
In the first standard example a 

In [3]:
class BankAccount:
    def __init__(self, account_name, account_number):
        self._account_name = account_name
        self._account_number = account_number
        self._account_balance = 0.0
    
    def get_balance(self):
        return self._account_balance
    def get_account_name(self):
        return self._account_name
    def withdraw(self, value):
        self._account_balance -= value
    def deposit(self, value):
        self._account_balance += value
    def show_balance(self):
        print(f"Balance is £{self._account_balance}")
    def __str__(self):
        return(f"The account {self._account_name} has £{self._account_balance:.2f} in it")

In [4]:
my_account = BankAccount("Phil", 122345)
my_account.deposit(100)
print(my_account)

The account Phil has £100.00 in it


When the class is instantiated, then python uses the type metaclass to create the BankAccount the EXPLICIT class definition looks like this :

In [10]:
class BankAccount(object, metaclass=type):
    def __init__(self, account_name, account_number):
        self._account_name = account_name
        self._account_number = account_number
        self._account_balance = 0.0
    
    def get_balance(self):
        return self._account_balance
    def get_account_name(self):
        return self._account_name
    def withdraw(self, value):
        self._account_balance -= value
    def deposit(self, value):
        self._account_balance += value
    def show_balance(self):
        print(f"Balance is £{self._account_balance}")
    def __str__(self):
        return(f"The account {self._account_name} has £{self._account_balance:.2f} in it")

In [11]:
my_account = BankAccount("Phil", 122345)
print(my_account)


The account Phil has £0.00 in it


Looking at the metaclass it is possible to implement another class instead of type, in the example below __new__ method will return a new class or a class object, it is a static method of object, the first argument passed is the thing to be made. In below mcs is metaclass, this must be followed by name in this case. __new__ is called first before __init__ sets up the class states. 

Note that  everything in python is an object, before we example metaclass lets prove that 

In [16]:
age = 35 
name = "phil"
print(age.__class__)
print(name.__class__)

print(age.__class__.__class__)
print(name.__class__.__class__)

<class 'int'>
<class 'str'>
<class 'type'>
<class 'type'>


Implementing metaclass, currently a basic implementation but this can be extended with *kwargs

In [18]:
class Human(type): 
    def __new__(mcs, name, bases, class_dict):
        class_=super().__new__(mcs,name,bases,class_dict)
        class_.eats_food = True
        return class_

class Person(object, metaclass=Human):
    def __init__(self, name, age):
        self.name = name
        self.age = age

    
pers = Person("Phil", 55)
pers.eats_food
            

True

    Looking at the previous but with kwargs

In [20]:
class Human(type): 
    def __new__(mcs, name, bases, class_dict, **kwargs):
        class_=super().__new__(mcs,name,bases,class_dict)
        if kwargs:
            for name, value in kwargs.items():
                setattr(class_, name, value)
        return class_

class Person(object, metaclass=Human, country="Great Britain", eats_food=True):
    def __init__(self, name, age):
        self.name = name
        self.age = age

    
pers = Person("Phil", 55)
print(pers.eats_food)
print(pers.country)


True
Great Britain


***Reviewing the mappingproxy***
mappingproxy - used to be dictproxy 
This is used in python to create an assurance and degree of constraint to a class level attributes. By default those attibutes for a class instance are exposed in a dictionary and mutable, for the class itself the values are exposed using mappingproxy ensuring that they are string values and that they are read-only and can only be set with the class.__setattr__ method



In [21]:
print(f"The class dict is {type(BankAccount.__dict__)}")
print(f"The class instance dict type is {type(my_account.__dict__)}")

The class dict is <class 'mappingproxy'>
The class instance dict type is <class 'dict'>


## Descriptors
The fuller code is shown: 

In [4]:
import sys
import inspect 

class TraceUse(object):
   
    def __init__(self, pname, pos):
        self.tpos = "%s line: %d" % (pname, pos)        
    def __get__(self, inst, cls):
        return self.tpos
    def __set__(self, inst, value):
        self.tpos = str(value)
    def __delete__(self, inst):
        pass


class Person(object):
    t1 = TraceUse('Initial',inspect.currentframe().f_lineno)
    
    def __init__(self, name):
        self.__name = str(name)
        # This gives the line number of the construction call
        Person.t1 = "%s line: %d" % ('__init__',
                     inspect.currentframe().f_lineno)
    def __str__(self):
        x = Person.t1
        return "%s %s" % (self.__name, x)


## Signatures
Let’s consider a scenario where you have written a very lengthy code and want to know the function call details. So what you can do is scroll through your code each and every time for different functions to know their details or you can work smartly. You can create a code where you can get the function details without scrolling through the code. This can be achieved in two ways –

Using signature() function
Using decorators

In [5]:
from inspect import signature

def myFunct(x:str, y:int):
	pass

t = signature(myFunct)

# print the signature of the function
print(t)

# print the annonation of the parameter
# of the function
print(t.parameters['x'])

# print the annonation of the parameter
# of the function
print(t.parameters['y'].annotation)


(x: str, y: int)
x: str
<class 'int'>
